## Import Library

In [1]:
import logging
from typing import Dict, Tuple
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
import random
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, roc_curve, precision_score
from sklearn.model_selection import train_test_split
from kedro.extras.datasets.pickle import PickleDataSet
from sklearn.model_selection import GridSearchCV
import xmltodict
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import sklearn



## Load/Save Data (XML/CSV)

In [2]:
def _clean_name (cadena):
    
    cadena_str= str(cadena)
    cadena_str=cadena_str[26:]
    cadena_str = cadena_str.replace('}', '',1)
    cadena_str = cadena_str.replace('\'','')
    
    return cadena_str

In [3]:
def load_from_xml (path):
    xml=open(path, encoding='ISO-8859-1')
    xmldict = xmltodict.parse(xml.read())
    df_enfermedades = pd.DataFrame()
    lista_enfer_sinto_prob=[]
    first_tree=xmldict["JDBOR"]["HPODisorderSetStatusList"]["HPODisorderSetStatus"]
    id=0
    for nodo in xmldict["JDBOR"]["HPODisorderSetStatusList"]["HPODisorderSetStatus"]:
     
        enfermedad=_clean_name(nodo["Disorder"]["Name"])
        sec_tree= nodo["Disorder"]["HPODisorderAssociationList"]
        if (len(sec_tree)==2):
            tam_sintomas=len(sec_tree["HPODisorderAssociation"])
            i=0
            registro_enfer_sinto_prob=[]
            while (i<tam_sintomas):
                registro_enfer_sinto_prob.append(enfermedad)
                registro_enfer_sinto_prob.append(id)
                registro_enfer_sinto_prob.append(sec_tree["HPODisorderAssociation"][i]["HPO"]["HPOTerm"])
                frecuencia=_clean_name(sec_tree["HPODisorderAssociation"][i]["HPOFrequency"]["Name"])
                registro_enfer_sinto_prob.append(frecuencia)
                lista_enfer_sinto_prob.append (registro_enfer_sinto_prob)
                registro_enfer_sinto_prob=[]
                i=i+1
        id=id+1
    df_enfermedades = pd.DataFrame(lista_enfer_sinto_prob)
    df_enfermedades = df_enfermedades.rename(columns={0:'Enfermedad',1:'Id_Enfermedad', 2:'Sintoma', 3:"Frecuencia"})
    
    return df_enfermedades        

In [4]:
def save_to_csv (data, path):
    data.to_csv(path, index=False)

In [5]:
def load_from_csv (path):
    data=pd.read_csv(path)
    return data   

## Exploratory Data Analysis (EDA)

In [275]:
def clean_data(data): 
    
    data=data.drop_duplicates()
    data=data.dropna()
    vc = data["Sintoma"].value_counts()
    vector=vc[vc < 50].index
    for a in vector:
        indexNames = data [ data["Sintoma"] == a ].index
        for b in indexNames:
            data.drop(b , inplace=True, axis=0)
  
    return data 

## Feature Engineering 

In [276]:
def _cambiar_columnas(df):
    
    columnas=len(df.columns)
    i=0
    while (i<columnas):
        df = df.rename(columns={df.columns[i]:i})
        i=i+1
        
        
    return df

In [409]:
def selection_and_preparation_data(data):
    
    #data=data[(data['Frecuencia']=="Muy frecuente (99-80%)")]
    data=data[(data['Frecuencia']=="Muy frecuente (99-80%)") |
              (data['Frecuencia']=="Frecuente (79-30%)") |
              (data['Frecuencia']=="Obligatorio (100%)") |
              (data['Frecuencia']=="Ocasional (29-5%)")
              
             ]
    #data=data[(data['Frecuencia']=="Frecuente (79-30%)")]

    
        
    return data


In [410]:
def generate_data_train (data, repeticiones):
    
    sintomas=data.iloc[:,1]
    sintomas_sin_repe=sintomas.drop_duplicates()
    sintomas_sin_repe=sintomas_sin_repe.sort_values(ascending
                              = True)
    df_train=pd.DataFrame(columns=sintomas_sin_repe)
   # df_train.insert(0, 'Enfermedad', 0)
    #df_train.insert(0, 'id_Enfermedad', 0)
 
    
    data_agrupado = (data.groupby("Enfermedad")
         .agg({"Sintoma": np.array, "Frecuencia": np.array})
         .reset_index()
         )
    
    z=0
    j=0
    while (z<repeticiones):
    
  #  print ("entra")
        i=0
        for a in data_agrupado["Enfermedad"]:
           # print ("Enfermedad: ", a)
        #vector_enfermedad.append(a)
        #lista=[]
            lst = [0] * ((len(sintomas_sin_repe)))
           
            df_train.loc[len(df_train)] = lst
           # df_train["Enfermedad"][j]=a
            #df_train["id_Enfermedad"][j]=(i)
           
            pos=0
            for b in data_agrupado["Sintoma"][i]:
                valor_aleatorio = random.random()
                frecuencia=data_agrupado["Frecuencia"][i][pos]
            #    print ("frecuencia:", frecuencia)
                if (frecuencia=="Muy frecuente (99-80%)"):
             #       print (b)
              #      print ("es muy frecuente")
                    #if (valor_aleatorio>0.4):
                        valor_entero=3
                    #else:
                     #   valor_entero=0
                elif (frecuencia=="Frecuente (79-30%)"):
               #     print (b)
                #    print ("es frecuente")
                    #if (valor_aleatorio>0.6):
                        valor_entero=2
                    #else:
                     #   valor_entero=0
                         
                elif (frecuencia=="Obligatorio (100%)"):
                    valor_entero=4

                elif (frecuencia=="Ocasional (29-5%)"):
                    valor_entero=1
                    
                df_train[b][j]=valor_entero
               
                pos=pos+1
            j=j+1
            i=i+1
        z=z+1
        print ("VUELTA: ", z)
        print ("Tamaño: ",j)
    return df_train      

## Train Data and Predict

In [279]:
def train_data (df_train):
    
    target=df_train["id_Enfermedad"]
   # data=data.drop (['Enfermedad'], axis=1)
    
    X_train, X_test, y_train, y_test = train_test_split(df_train,target)
    rf= RandomForestClassifier()
    rf.fit (X_train,y_train)
    #prediccion=rf.predict(X_test)
    
    return df_train,rf,target,X_test,y_test, X_train, y_train


In [280]:
def aparece_y_como (scoring_enfermedades,id_Sintoma, df_Enfermedades):
    

    
    j=0
    enfermedades=[]
    while (j<len(scoring_enfermedades)):
        enfermedad=[]
        id_enfermedad=scoring_enfermedades["index"][j]
        scoring=scoring_enfermedades[id_Sintoma][j]
        
        enfermedad.append(id_enfermedad)
       
        enfermedad.append(df_Enfermedades[df_Enfermedades["index"]==id_enfermedad]["Enfermedad"].values[0])
   
        enfermedad.append(scoring)
        lista=df_EnfeySinto_select[df_EnfeySinto_select["Enfermedad"]==
                                   df_Enfermedades.loc[id_enfermedad][1]]
        lista=lista.reset_index()
        sintoma= df_Sintomas.loc[id_Sintoma].Sintoma
        
        i=0
        while i<len(lista):
         
            if lista["Sintoma"][i]==sintoma:
                enfermedad.append(lista["Frecuencia"][i])
            
            i=i+1  
        j=j+1
        enfermedades.append(enfermedad)
    return enfermedades

In [281]:
def saca_enfermedades (vector,df_Enfermedades):
    
    enfermedades=[]
    for j in vector:
        enfermedades.append(df_Enfermedades[df_Enfermedades["index"]==j]["Enfermedad"])
        
    return enfermedades

In [353]:


#common = set(first).intersection(second)
##HACE INTERSECCIÓN DE VARIOS SINTOMAS
def trata_sintomas (sintomas,df_transpuesta):
    comunes=[]
    primera_iter=True
    for i in sintomas:
        vector=[]
        lista,vector=predict_similitud_entre_usuarios_by_pearson(df_transpuesta,i,20)
        
        v=vector.reset_index()
        
        sintoma=v["index"]
        sintoma=list(sintoma)
      
        if primera_iter:
            comunes=sintoma
            primera_iter=False

        else:            
            comunes = set(comunes).intersection(sintoma)
        
    return comunes


In [303]:
def predict_collaborative_filtering_ser_based(ratings, sintoma, elementos):
 
    id_sintoma = df_Sintomas[df_Sintomas['Sintoma'] == sintoma].index.values[0]

    ratings_train, ratings_test = train_test_split(ratings, test_size = 0.2, shuffle=False, random_state=42)
    print (ratings_train.shape)
    print (ratings_test.shape)
    sim_matrix = 1 - sklearn.metrics.pairwise.cosine_distances(ratings)
    
    #Matriz de similitud entre los usuarios (distancia del coseno -vectores-).
    #Predecir la valoración desconocida de un ítem i para un usuario activo u basandonos en la suma ponderada de
    #todas las valoraciones del resto de usuarios para dicho ítem.
    #Recomendaremos los nuevos ítems a los usuarios según lo establecido en los pasos anteriores.
    #separar las filas y columnas de train y test
    sim_matrix_train = sim_matrix[0:386,0:386]
    sim_matrix_test = sim_matrix[386:483,386:483]
    
    users_predictions = sim_matrix_train.dot(ratings_train) / np.array([np.abs(sim_matrix_train).sum(axis=1)]).T
    
    
    
    #Predicciones (las recomendaciones!)
    
    user0=users_predictions.argsort()[id_sintoma]
    vector_id_enfermedad_scoring=[]
    for i, aRepo in enumerate(user0[-elementos:]):
        v=[]
        selRepo = df_Enfermedades[df_Enfermedades["index"]==aRepo]
  
       # print('Enfermedad:', selRepo["Enfermedad"] , 'scoring:', users_predictions[sintoma_ver][aRepo])
        v.append (aRepo)
        v.append (users_predictions[id_sintoma][aRepo])
        vector_id_enfermedad_scoring.append(v)
        
    vector_id_enfermedad_scoring=pd.DataFrame(vector_id_enfermedad_scoring)
    vector_id_enfermedad_scoring = vector_id_enfermedad_scoring.rename(columns={1:id_sintoma, 0:"index"})
    
    listado_completo=aparece_y_como (vector_id_enfermedad_scoring,id_sintoma, df_Enfermedades)
    #listado_completo= list(reserved(listado_completo))
    return listado_completo,users_predictions,ratings_train, ratings_test, sim_matrix

In [445]:
def predict_similitud_entre_usuarios_by_pearson (df_train_generated,sintoma,elementos):
    
    id_sintoma = df_Sintomas[df_Sintomas['Sintoma'] == sintoma].index.values[0]

    
    df_transpuesta=df_train_generated
    correlaciones = df_transpuesta.corr(method='pearson')
    #correlaciones[34].sort_values(ascending=False)
    #Top = 3
    # Top relaciones positivas
    #correlaciones[0].sort_values(ascending=False)[1:].head(Top)
    #correlaciones[0].sort_values(ascending=True).head(Top)
    MediasUsers = df_transpuesta.mean(numeric_only=True,skipna=True)
    DesviacionesUsers = df_transpuesta.std(numeric_only=True,skipna=True)
    df_ValoracionesNorm = df_transpuesta[MediasUsers.index].sub(MediasUsers, axis='columns')
   # df_ValoracionesNorm.sort_values(0).head(10)
    df_ValoracionesNorm = df_ValoracionesNorm.div(DesviacionesUsers, axis='columns')
    #df_ValoracionesNorm.sort_values(0).head(10)
    Numerador = df_ValoracionesNorm.dot(correlaciones)
    Numerador.sort_values(0,ascending=False)
    Denominador = correlaciones.abs().sum()
    Cociente = Numerador.div(Denominador, axis='columns')
   # Cociente.sort_values(0,ascending=False)
    Valoraciones = Cociente.mul(DesviacionesUsers,axis='columns').add(MediasUsers, axis='columns')
    #Valoraciones.sort_values(0,ascending=False)
    a=Valoraciones[id_sintoma].sort_values(ascending=False).head(elementos)
    vector=pd.DataFrame(a)
    vector=vector.reset_index()
    lista=aparece_y_como (vector,id_sintoma, df_Enfermedades)
    k = 5
    neighbors = NearestNeighbors(n_neighbors=k, metric='cosine')
    neighbors.fit(Valoraciones)
    top_k_distances, top_k_users = model.kneighbors(Valoraciones, return_distance=True)
 #top_k_distances.shape
    res = top_k_users[id_sintoma]
    
    #return lista,a,sintoma_ver,vector
    
    return lista,a, res

In [285]:
#def rank (df_train,rf,X_test):
  #  predicciones = rf.predict_proba(X_test)
   # predicciones[:5, :]
   # df_rank = pd.DataFrame(data=predicciones, columns=['0', '1','2','3','4', '5', '6', '7', '8'])
   # df_rank['clasificacion_default_0.5'] = np.where(df_predicciones['0'] > df_predicciones['1'], 0, 1)
   # return df_rank
    

## Evaluate

In [286]:

def get_mse(preds, actuals):
    if preds.shape[1] != actuals.shape[1]:
        actuals = actuals.T
    preds = preds[actuals.nonzero()].flatten()
    actuals = actuals[actuals.nonzero()].flatten()
    return mean_squared_error(preds, actuals)
 


In [287]:
def evaluate (classifier, X_test, y_test):

    y_pred=classifier.predict(X_test)
    probs=classifier.predict_proba(X_test) [:, 1]
    
    acc = accuracy_score(y_test, y_pred)
    pres= precision_score (y_test, y_pred, average='micro')
    f1 = f1_score(y_test, y_pred, average='micro')
    
   # roc_auc = roc_auc_score(y_test,probs, multi_class='ovr')
    #auc = roc_auc_score(y_test, probs)
    print ("Accurancy: ", acc)
    print ("Precision: ", pres)
    print ("f1: ", f1)
    #print ("auc: ", auc)
    
   # fpr, tpr, threshols = roc_curve(y_test, probs)
   # plt.plot(fpr,tpr)
   # plt.xlabel('False positives rate')
   # plt.ylabel('True positives rate')
   # plt.gcf().set_size_inches(8,6)
   

## Notebook Main: Carga de Datos de DataFrames

In [288]:
df_EnfeySinto=load_from_xml("../data/01_raw/enfermedades.xml")

In [289]:
df_EnfeySinto["Frecuencia"].unique()

array(['Muy frecuente (99-80%)', 'Frecuente (79-30%)',
       'Ocasional (29-5%)', 'Muy poco frecuente (4-1%)',
       'Excluyente (0%)', 'Obligatorio (100%)'], dtype=object)

In [290]:
print ("Después de filtrado por sintomas filas")
print ("Enfermedades: ", df_EnfeySinto["Enfermedad"].nunique())
print ("Sintomas: ", df_EnfeySinto["Sintoma"].nunique())
print ("Frecuencias: ", df_EnfeySinto["Frecuencia"].nunique())  

Después de filtrado por sintomas filas
Enfermedades:  4262
Sintomas:  8303
Frecuencias:  6


In [291]:
df_EnfeySinto_cleaned=clean_data(df_EnfeySinto)

In [292]:
print ("Después de filtrado por sintomas filas")
print ("Enfermedades: ", df_EnfeySinto_cleaned["Enfermedad"].nunique())
print ("Sintomas: ", df_EnfeySinto_cleaned["Sintoma"].nunique())
print ("Frecuencias: ", df_EnfeySinto_cleaned["Frecuencia"].nunique())  

Después de filtrado por sintomas filas
Enfermedades:  4187
Sintomas:  483
Frecuencias:  6


In [411]:
df_EnfeySinto_select=selection_and_preparation_data(df_EnfeySinto_cleaned)
df_EnfeySinto_select

,Enfermedad,Id_Enfermedad,Sintoma,Frecuencia
0,Enfermedad de Alexander,0,Macrocephaly,Muy frecuente (99-80%)
1,Enfermedad de Alexander,0,Intellectual disability,Muy frecuente (99-80%)
2,Enfermedad de Alexander,0,Seizure,Muy frecuente (99-80%)
3,Enfermedad de Alexander,0,Spasticity,Muy frecuente (99-80%)
4,Enfermedad de Alexander,0,Agenesis of corpus callosum,Muy frecuente (99-80%)
...,...,...,...,...
112669,Artrogriposis múltiple congénita miogénica aut...,4262,Depressed nasal bridge,Ocasional (29-5%)
112671,Artrogriposis múltiple congénita miogénica aut...,4262,Postnatal growth retardation,Ocasional (29-5%)
112672,Artrogriposis múltiple congénita miogénica aut...,4262,Gastrostomy tube feeding in infancy,Ocasional (29-5%)
112679,Nevo de cabello lanoso,4263,Fine hair,Muy frecuente (99-80%)


In [412]:
  print ("Después de borrado filas, duplicates y NAN")
  print ("Enfermedades: ", df_EnfeySinto_select["Enfermedad"].nunique())
  print ("Sintomas: ", df_EnfeySinto_select["Sintoma"].nunique())
  print ("Frecuencias: ", df_EnfeySinto_select["Frecuencia"].nunique())  


Después de borrado filas, duplicates y NAN
Enfermedades:  4173
Sintomas:  483
Frecuencias:  4


In [413]:
df_EnfeySinto_select=df_EnfeySinto_select.drop("Id_Enfermedad", axis=1)
df_EnfeySinto_select

,Enfermedad,Sintoma,Frecuencia
0,Enfermedad de Alexander,Macrocephaly,Muy frecuente (99-80%)
1,Enfermedad de Alexander,Intellectual disability,Muy frecuente (99-80%)
2,Enfermedad de Alexander,Seizure,Muy frecuente (99-80%)
3,Enfermedad de Alexander,Spasticity,Muy frecuente (99-80%)
4,Enfermedad de Alexander,Agenesis of corpus callosum,Muy frecuente (99-80%)
...,...,...,...
112669,Artrogriposis múltiple congénita miogénica aut...,Depressed nasal bridge,Ocasional (29-5%)
112671,Artrogriposis múltiple congénita miogénica aut...,Postnatal growth retardation,Ocasional (29-5%)
112672,Artrogriposis múltiple congénita miogénica aut...,Gastrostomy tube feeding in infancy,Ocasional (29-5%)
112679,Nevo de cabello lanoso,Fine hair,Muy frecuente (99-80%)


In [414]:
df_Sintomas=df_EnfeySinto_select.groupby (["Sintoma"]).count().reset_index()
df_Sintomas=df_Sintomas.drop(["Enfermedad","Frecuencia"], axis=1)
df_Sintomas

,Sintoma
0,Abdominal distention
1,Abdominal pain
2,Abnormal bleeding
3,Abnormal blistering of the skin
4,Abnormal cardiac septum morphology
...,...
478,Weight loss
479,Wide intermamillary distance
480,Wide mouth
481,Wide nasal bridge


In [415]:
df_Enfermedades=df_EnfeySinto_select.groupby (["Enfermedad"]).count().reset_index()
df_Enfermedades=df_Enfermedades.drop(["Sintoma","Frecuencia"], axis=1)
df_Enfermedades=df_Enfermedades.reset_index()
df_Enfermedades

,index,Enfermedad
0,0,AICA ribosiduria
1,1,ALG1-CDG
2,2,ALG11-CDG
3,3,ALG12-CDG
4,4,ALG13-CDG
...,...,...
4168,4168,Xeroderma pigmentoso
4169,4169,Xeroderma pigmentoso tipo variante
4170,4170,Zigomicosis
4171,4171,síndrome de talla baja-sordera-disfunción neut...


## Datos Entrenamiento y Matrix

In [416]:
%%time
df_train_generated=generate_data_train (df_EnfeySinto_select, 1)

VUELTA:  1
Tamaño:  4173
CPU times: user 6.22 s, sys: 42.9 ms, total: 6.26 s
Wall time: 6.26 s


In [417]:
df_train_generated

Sintoma,Abdominal distention,Abdominal pain,Abnormal bleeding,Abnormal blistering of the skin,Abnormal cardiac septum morphology,Abnormal cerebral white matter morphology,Abnormal dermatoglyphics,Abnormal enzyme/coenzyme activity,Abnormal facial shape,Abnormal fingernail morphology,...,Visual impairment,Visual loss,Vomiting,Waddling gait,Webbed neck,Weight loss,Wide intermamillary distance,Wide mouth,Wide nasal bridge,Wide nose
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3,0,0
1,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,4,1,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4168,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4169,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4170,0,1,0,1,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
4171,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [418]:
df_train_generated=_cambiar_columnas(df_train_generated)
df_train_generated

Sintoma,0,1,2,3,4,5,6,7,8,9,...,473,474,475,476,477,478,479,480,481,482
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3,0,0
1,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,4,1,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4168,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4169,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4170,0,1,0,1,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
4171,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [419]:
df_matrix=df_train_generated.transpose()
df_matrix

,0,1,2,3,4,5,6,7,8,9,...,4163,4164,4165,4166,4167,4168,4169,4170,4171,4172
Sintoma,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
479,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
480,3,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


## Collaborative Filtering: User based

In [420]:
listado_completo,users_predictions,ratings_train, ratings_test, sim_matrix=predict_collaborative_filtering_ser_based(
    df_matrix.values, "Photophobia",10)

(386, 4173)
(97, 4173)


In [421]:
listado_completo

[[2724, 'Síndrome de Jalili', 0.553900622368422, 'Muy frecuente (99-80%)'],
 [96,
  'Albinismo oculocutáneo tipo 1A',
  0.5563920640894137,
  'Muy frecuente (99-80%)'],
 [423, 'Cistinosis', 0.5585931757971797, 'Muy frecuente (99-80%)'],
 [94, 'Albinismo oculo-cutáneo', 0.5710903650989095, 'Muy frecuente (99-80%)'],
 [1939, 'Mucolipidosis tipo IV', 0.5811981856261854, 'Muy frecuente (99-80%)'],
 [2708,
  'Síndrome de Hermansky-Pudlak',
  0.5838646553679144,
  'Frecuente (79-30%)'],
 [101,
  'Albinismo oculocutáneo tipo 5',
  0.6180678664196122,
  'Obligatorio (100%)'],
 [2324, 'Retinosis pigmentaria', 0.655322038459253, 'Muy frecuente (99-80%)'],
 [3470,
  'Síndrome de ictiosis folicular-alopecia-fotofobia',
  0.6916799958704494,
  'Muy frecuente (99-80%)'],
 [1044, 'Enfermedad de Behçet', 0.7292557908579722, 'Muy frecuente (99-80%)']]

In [422]:
get_mse(users_predictions, ratings_train)

3.0554792369867765

In [423]:
# Realizo las predicciones para el test set
users_predictions_test = sim_matrix.dot(df_matrix.values) / np.array([np.abs(sim_matrix).sum(axis=1)]).T
users_predictions_test = users_predictions_test[386:483,:]
 
get_mse(users_predictions_test, ratings_test)

3.098369946008622

In [425]:
df_EnfeySinto_select[df_EnfeySinto_select["Enfermedad"]=="Albinismo oculocutáneo tipo 1A"]

,Enfermedad,Sintoma,Frecuencia
30096,Albinismo oculocutáneo tipo 1A,Visual impairment,Frecuente (79-30%)
30098,Albinismo oculocutáneo tipo 1A,Photophobia,Muy frecuente (99-80%)
30099,Albinismo oculocutáneo tipo 1A,Nystagmus,Muy frecuente (99-80%)
30101,Albinismo oculocutáneo tipo 1A,Hyperkeratosis,Ocasional (29-5%)
30102,Albinismo oculocutáneo tipo 1A,Hypopigmentation of the skin,Muy frecuente (99-80%)
30104,Albinismo oculocutáneo tipo 1A,Thickened skin,Ocasional (29-5%)
30108,Albinismo oculocutáneo tipo 1A,Hypopigmentation of hair,Muy frecuente (99-80%)


mean squared error

## Similitud entre Usuarios: Correlación de Pearson

SOLO UN SINTOMA

In [446]:
listado_completo, vector_enfermedades_scoring, model=predict_similitud_entre_usuarios_by_pearson(
    df_train_generated,"Photophobia", 10)


In [447]:
listado_completo

[[101,
  'Albinismo oculocutáneo tipo 5',
  0.5845658288799815,
  'Obligatorio (100%)'],
 [94, 'Albinismo oculo-cutáneo', 0.5725874709713061, 'Muy frecuente (99-80%)'],
 [1044, 'Enfermedad de Behçet', 0.5455766773484193, 'Muy frecuente (99-80%)'],
 [95,
  'Albinismo oculocutáneo tipo 1',
  0.5274315437182189,
  'Muy frecuente (99-80%)'],
 [96,
  'Albinismo oculocutáneo tipo 1A',
  0.5017100124867448,
  'Muy frecuente (99-80%)'],
 [2593,
  'Síndrome de Chédiak-Higashi',
  0.48872445367614276,
  'Frecuente (79-30%)'],
 [2708,
  'Síndrome de Hermansky-Pudlak',
  0.48553495038836814,
  'Frecuente (79-30%)'],
 [2324, 'Retinosis pigmentaria', 0.4847117510401041, 'Muy frecuente (99-80%)'],
 [100,
  'Albinismo oculocutáneo tipo 4',
  0.4745922978532195,
  'Frecuente (79-30%)'],
 [2724, 'Síndrome de Jalili', 0.4723351533743384, 'Muy frecuente (99-80%)']]

In [448]:
vector_enfermedades_scoring

101     0.584566
94      0.572587
1044    0.545577
95      0.527432
96      0.501710
2593    0.488724
2708    0.485535
2324    0.484712
100     0.474592
2724    0.472335
Name: 360, dtype: float64

In [433]:
df_EnfeySinto_select[(df_EnfeySinto_select["Enfermedad"]=="Linfoma de Hodgkin")]

,Enfermedad,Sintoma,Frecuencia
32720,Linfoma de Hodgkin,Hyperhidrosis,Frecuente (79-30%)
32721,Linfoma de Hodgkin,Pruritus,Frecuente (79-30%)
32722,Linfoma de Hodgkin,Ataxia,Ocasional (29-5%)
32723,Linfoma de Hodgkin,Splenomegaly,Ocasional (29-5%)
32724,Linfoma de Hodgkin,Weight loss,Frecuente (79-30%)
32726,Linfoma de Hodgkin,Fever,Frecuente (79-30%)
32727,Linfoma de Hodgkin,Anorexia,Frecuente (79-30%)
32728,Linfoma de Hodgkin,Dyspnea,Ocasional (29-5%)
32729,Linfoma de Hodgkin,Hemoptysis,Ocasional (29-5%)
32730,Linfoma de Hodgkin,Hepatomegaly,Ocasional (29-5%)


VAMOS CON MULTI-SINTOMA

In [428]:
SINTOMAS_EJEMPLO = ['Chest pain','Weight loss','Cough']

In [429]:
sintomas_comunes=trata_sintomas (SINTOMAS_EJEMPLO,df_train_generated)
sintomas_comunes

{1421, 1696, 1697}

In [430]:
print(saca_enfermedades (sintomas_comunes,df_Enfermedades))

[1696    Linfoma de Hodgkin
Name: Enfermedad, dtype: object, 1697    Linfoma de Hodgkin clásico
Name: Enfermedad, dtype: object, 1421    Granulomatosis con poliangeítis
Name: Enfermedad, dtype: object]


In [399]:
sintoma="Photophobia"

In [400]:
id_sintoma = df_Sintomas[df_Sintomas['Sintoma'] == sintoma].index.values[0]
id_sintoma

360

NearestNeighbors(metric='cosine')

In [438]:
top_k_distances, top_k_users = model.kneighbors(ratings_train, return_distance=True)
top_k_distances.shape

ValueError: X has 4173 features, but NearestNeighbors is expecting 483 features as input.

In [403]:
top_k_distances[id_sintoma]


array([0.        , 0.77884975, 0.85882109, 0.86390533, 0.86512184])

In [404]:
top_k_users.shape

(386, 5)

In [405]:
res = top_k_users[id_sintoma]
res

array([360, 333, 244,  21, 119])

In [406]:
users_predicts_k = np.zeros(ratings_train.shape)
for i in range(ratings_train.shape[0]):# para cada usuario del conjunto de entrenamiento
    users_predicts_k[i,:] = top_k_distances[i].T.dot(ratings_train[top_k_users][i]) / np.array([np.abs(top_k_distances[i].T).sum(axis=0)]).T
     

In [407]:
user0=users_predicts_k.argsort()[id_sintoma]
# los tres con mayor puntaje en la predic para este usuario
# agrego 4 puesto que uno es mi propio repositorio
for aRepo in (user0[-4:]):
    selRepo = df_Enfermedades[df_Enfermedades['index'] == aRepo]
    print(selRepo['Enfermedad'], 'puntaje:', users_predicts_k[id_sintoma][aRepo])

96    Albinismo oculocutáneo tipo 4
Name: Enfermedad, dtype: object puntaje: 1.2309780199455262
90    Albinismo oculo-cutáneo
Name: Enfermedad, dtype: object puntaje: 1.2317006933688752
2879    Síndrome de Vici
Name: Enfermedad, dtype: object puntaje: 1.2547316616150224
93    Albinismo oculocutáneo tipo 1B
Name: Enfermedad, dtype: object puntaje: 1.2547316616150224


In [ ]:
 user0=users_predictions.argsort()[id_sintoma]
    vector_id_enfermedad_scoring=[]
    for i, aRepo in enumerate(user0[-elementos:]):
        v=[]
        selRepo = df_Enfermedades[df_Enfermedades["index"]==aRepo]
  
       # print('Enfermedad:', selRepo["Enfermedad"] , 'scoring:', users_predictions[sintoma_ver][aRepo])
        v.append (aRepo)
        v.append (users_predictions[id_sintoma][aRepo])
        vector_id_enfermedad_scoring.append(v)

In [396]:
get_mse(users_predicts_k, ratings_train)
     

1.475195256059977

In [397]:
get_mse(users_predicts_k, ratings_test)


2.1039563748166334

In [408]:
df_EnfeySinto_select[(df_EnfeySinto_select["Enfermedad"]=="Síndrome de Vici")]

,Enfermedad,Sintoma,Frecuencia
15418,Síndrome de Vici,High palate,Frecuente (79-30%)
15422,Síndrome de Vici,Cataract,Frecuente (79-30%)
15424,Síndrome de Vici,Nystagmus,Frecuente (79-30%)
15425,Síndrome de Vici,Optic atrophy,Frecuente (79-30%)
15426,Síndrome de Vici,Hypopigmentation of the skin,Muy frecuente (99-80%)
15428,Síndrome de Vici,Intellectual disability,Muy frecuente (99-80%)
15429,Síndrome de Vici,Seizure,Frecuente (79-30%)
15430,Síndrome de Vici,Hypotonia,Muy frecuente (99-80%)
15431,Síndrome de Vici,Global developmental delay,Muy frecuente (99-80%)
15432,Síndrome de Vici,Agenesis of corpus callosum,Muy frecuente (99-80%)


In [ ]:
import pickle

#pickle.dump(model,open("modelo19m.pickle","wb"))
model2=pickle.load(open("modelo19m.pickle","rb"))

In [ ]:
df_train_generated.to_csv("df_train_final.csv", index=False)

In [ ]:
from joblib import dump
dump(model, 'modelo19m.joblib') 

In [ ]:
from joblib import load
new_model = load('modelo19m.joblib')

In [ ]:
evaluate (new_model, X_test, y_test)

In [ ]:
param_grid = { 
    'n_estimators': [200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}

In [ ]:
CV_rfc = GridSearchCV(estimator=model, param_grid=param_grid, cv= 5)
CV_rfc.fit(X_train, y_train)

In [ ]:
CV_rfc.best_params_

In [ ]:
X_test

In [ ]:
X_test.iloc[[1]]

In [ ]:
model.predict(X_test.iloc[[2]])

In [ ]:
y_test.iloc[[2]]

In [ ]:
predicciones = model.predict_proba(X_test.iloc[[2]])
predicciones.shape


In [ ]:
X_test.iloc[[2]]


In [ ]:
a=df_train_generated.columns


In [ ]:
b=a.to_list()
b.remove('id_Enfermedad')
b

In [ ]:
df_train_generated.columns
df_predicciones = pd.DataFrame(data=predicciones)
#df_predicciones['clasificacion_default_0.5'] = np.where(df_predicciones['0'] > df_predicciones['1'], 0, 1)
df_predicciones

In [ ]:
df_predicciones2=df_predicciones.transpose()
df_predicciones2

In [ ]:
list_to_rank=df_predicciones2[df_predicciones2[0]>0]

In [ ]:
rank = list_to_rank.sort_values(0,ascending=False)
rank

In [ ]:
rank2=rank.reset_index()

In [ ]:
rank2=rank2.head(5)
rank2

In [ ]:
rank2.merge (data)

In [ ]:
rank2["Enfermedad"]= rank2.apply(lambda row:
                                 data[data['Id_Enfermedad']==row["index"]].iloc[0][0]
                                 if data['Id_Enfermedad']==row["index"]
                                 else 0,
                                 axis=1
                                )

In [ ]:
data.iloc[0][0]

In [ ]:
for i in range(1):
    
    a=data[data['Id_Enfermedad']==rank2["index"][i]].iloc[0][0]
    a

In [ ]:
a

In [ ]:
a.iloc[0][0]

In [ ]:
list_rank=rank.to_numpy().tolist()
list_rank


In [ ]:
for i in range(5):
    print (i)
    print (lis)
        

In [ ]:
data[data['Id_Enfermedad']==2389]

#### 